# 🎨 Fooocus Batch Processor - Google Colab

Ez a notebook lehetővé teszi, hogy TXT fájlokból automatikusan generálj képeket a Fooocus-szal, 32-es batch-ekben.

## 📋 Használat

1. **Futtasd az összes cellát sorban**
2. **Töltsd fel a TXT fájljaidat** (egy sor = egy prompt)
3. **Várd meg a generálást**
4. **Töltsd le a képeket**

## 1️⃣ Telepítés és Inicializálás

In [ ]:
# Klónozás és telepítés
!git clone https://github.com/[YOUR-USERNAME]/Fooocus.git
%cd Fooocus

# Függőségek telepítése
!pip install -q -r requirements_versions.txt

print("✓ Telepítés kész!")

## 2️⃣ Fooocus Indítása (háttérben)

In [ ]:
import subprocess
import time
import os

# Fooocus indítása háttérben
print("Fooocus indítása...")
process = subprocess.Popen(
    ['python', 'launch.py', '--share', '--always-high-vram'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Várunk, amíg elindul
print("Várakozás az indulásra (ez eltarthat 1-2 percig)...")
time.sleep(60)

print("✓ Fooocus elindult!")

## 3️⃣ TXT Fájlok Feltöltése

Töltsd fel a TXT fájljaidat! Formátum:
- **Egy sor = egy prompt**
- `#`-tel kezdődő sorok = megjegyzések (ignorálva)

In [ ]:
from google.colab import files
import os

# Prompts mappa létrehozása
os.makedirs('prompts', exist_ok=True)

print("Töltsd fel a TXT fájljaidat:")
uploaded = files.upload()

# Fájlok áthelyezése a prompts mappába
for filename in uploaded.keys():
    if filename.endswith('.txt'):
        os.rename(filename, f'prompts/{filename}')
        print(f"✓ {filename} feltöltve")

print("\n✓ Minden fájl feltöltve!")

## 4️⃣ Batch Generálás Indítása

In [ ]:
# Batch processor futtatása
!python batch_processor.py --batch-size 32

print("\n✓ Generálás befejezve!")

## 5️⃣ Generált Képek Letöltése

In [ ]:
import shutil
from pathlib import Path

# Outputs mappa tömörítése
print("Képek tömörítése...")
shutil.make_archive('generated_images', 'zip', 'outputs')

# Letöltés
print("Letöltés indítása...")
files.download('generated_images.zip')

print("\n✓ Képek letöltve! (generated_images.zip)")

## 🔧 Opcionális: Egyéni Beállítások

In [ ]:
# Kisebb batch méret (ha memória problémák vannak)
!python batch_processor.py --batch-size 16

# TXT fájlok megtartása (ne törölje őket)
!python batch_processor.py --keep-files

# Egyéni prompts mappa
!python batch_processor.py --prompts-dir my_prompts

## 📊 Statisztikák

In [ ]:
from pathlib import Path

# Generált képek száma
output_dir = Path('outputs')
if output_dir.exists():
    images = list(output_dir.glob('**/*.png')) + list(output_dir.glob('**/*.jpg'))
    print(f"📊 Összesen {len(images)} kép generálva")
    
    # Méret
    total_size = sum(img.stat().st_size for img in images)
    print(f"💾 Teljes méret: {total_size / 1024 / 1024:.2f} MB")
else:
    print("Még nincsenek generált képek")

---

## 💡 Tippek

- **Memória problémák**: Csökkentsd a `--batch-size` értékét (pl. 8 vagy 16)
- **Gyorsabb generálás**: Használj egyszerűbb promptokat
- **Több prompt**: Készíts több TXT fájlt és töltsd fel őket egyszerre

## 🐛 Hibaelhárítás

- **"CUDA out of memory"**: Csökkentsd a batch méretet
- **"No TXT files found"**: Ellenőrizd, hogy feltöltötted-e a fájlokat
- **Lassú generálás**: Ez normális, a Fooocus minőségi képeket készít

---

**Készítette**: Fooocus Batch Processor  
**GitHub**: [YOUR-USERNAME]/Fooocus